In [17]:
from rdflib import Graph, URIRef, Namespace
from api.src.schemas.authorities.authority import Authority
from api.src.function.authorities.makeGraph import MakeGraph
from api.src.function.authorities.generateID import GenerateId
from pyfuseki import FusekiUpdate

In [3]:
g = Graph()
# g.parse('http://id.loc.gov/authorities/subjects/sh85076502.madsrdf.rdf')
g.parse('http://id.loc.gov/authorities/subjects/sh85076502.rdf')
# g.serialize('libraries-SKOS.ttl')


<Graph identifier=Nbf180feab6c04a799f3d2c4180ea0bc4 (<class 'rdflib.graph.Graph'>)>

In [3]:
uri = URIRef('http://id.loc.gov/authorities/subjects/sh85076502')

In [6]:
uri = 'http://id.loc.gov/authorities/subjects/sh85076502'
prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""
qlccn = f"""{prefix}
SELECT ?o WHERE {{ 
    <{uri}> identifiers:lccn ?o }}"""

qType = f"""{prefix}
SELECT ?o WHERE {{ 
    <{uri}> rdf:type ?o }}"""

qElementList = f"""{prefix}
SELECT ?elementValue ?type WHERE  {{
	<{uri}> madsrdf:elementList ?o .
  ?o rdf:rest* ?node .
  ?node rdf:first ?e .
  ?e madsrdf:elementValue ?elementValue .
  ?e rdf:type ?type
  }}"""

qComponentList = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:componentList ?componentList .
  }}"""

qFullerName = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:fullerName ?fullerName .
  }}"""

qBroader = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasBroaderAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qCloseExternal = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasCloseExternalAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qExactExternal = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasExactExternalAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qNarrower = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasNarrowerAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""





In [4]:
def getUri(obj, metadado, authority, graph):
    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadado} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        uris = list()
        for i in response.bindings:
          uri = {
                  "value": i.get('value').n3().replace('<','').replace('>',''),
              "label": {
                "value": i.get('label').value,
                "lang": i.get('label').language
              } }
          uris.append(uri)
        obj[metadado] = uris
    return obj

obj = {}
uri = 'http://id.loc.gov/authorities/subjects/sh85076502'
obj = getUri(obj, "hasBroaderAuthority", uri, g)
obj


{'hasBroaderAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85038731',
   'label': {'value': 'Documentation', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85108685',
   'label': {'value': 'Public institutions', 'lang': 'en'}}]}

In [15]:
obj = {
    'type': [i.split("#")[1] for [i] in g.query(qType) ][0],
    "adminMetadata": {
    "generationProcess": {
      "label": "BiblioKeia Beta"
    },
    "identifiedBy": {
      "assigner": "http://id.loc.gov/vocabulary/organizations/brmninpa",
      "value": id
    }
  }, 
  "isMemberOfMADSCollection": "http://id.loc.gov/authorities/subjects/"
    }

[obj['lccn']] = [i[0].value for i in g.query(qlccn) ]

# elementList
response = g.query(qElementList)
if len(response.bindings) > 0:
    elementList = list()
    for i in response.bindings:
      element = {
          "type": i.get('type').split("#")[1],
        "elementValue": {
          "value":  i.get('elementValue').value,
          "lang": i.get('elementValue').language
        }
      }
      elementList.append(element)
    obj['elementList'] = elementList
  
# ComponentList
response = g.query(qComponentList)
if len(response.bindings) > 0:
    print("qComponentList")

# fullerName
response = g.query(qComponentList)
if len(response.bindings) > 0:
    print("fullerName")

# birthDate: Optional[str]
#     birthPlace: Optional[Uri]
#     deathDate: Optional[str]
#     occupation: Optional[list[Uri]]
#     hasAffiliation: Optional[list[Affiliation]]

# hasBroaderAuthority
obj = getUri(obj, "hasBroaderAuthority", uri, g)

# hasCloseExternalAuthority
obj = getUri(obj, "hasCloseExternalAuthority", uri, g)

# hasExactExternalAuthority
obj = getUri(obj, "hasExactExternalAuthority", uri, g)

# hasNarrowerAuthority
obj = getUri(obj, "hasExactExternalAuthority", uri, g)

# hasVariant: 




request = Authority(**obj)
request
# obj

Authority(type='Topic', identifiers=None, adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 5, 2), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess=GenerationProcess(label='BiblioKeia Beta', generationDate='2023-05-02T19:56:44'), identifiedBy=IdentifiedBy(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='bkau-1'), status=Status(value='mstatus:new', label='novo')), elementList=[Element(type='TopicElement', elementValue=Label(value='Libraries', lang='en'))], componentList=None, fullerName=None, birthDate=None, birthPlace=None, deathDate=None, occupation=None, hasAffiliation=None, hasBroaderAuthority=[Uri(value='http://id.loc.gov/authorities/subjects/sh85038731', label=Label(value='Documentation', lang='en')), Uri(value='http://id.loc.gov/authorities/subjects/sh85108685', label=Label(va

In [16]:
graph = MakeGraph(request, id)
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

In [18]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(graph)
response

In [20]:
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [126]:
response = g.query(qElementList)
for i in response.bindings:
    element = {
         "type": i.get('type').split("#")[1],
      "elementValue": {
        "value":  i.get('elementValue').value,
        "lang": i.get('elementValue').language
      }
    }
    print(element)

{'type': 'TopicElement', 'elementValue': {'value': 'Libraries', 'lang': 'en'}}


In [125]:
x = i.get('type').split("#")[1]
x

'TopicElement'

In [84]:
graph = MakeGraph(request, 'bk-TESTE')
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

In [12]:
for i in g.triples((uri, None, None)):
    print(i)

(rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85076502'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#Topic'))
(rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85076502'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#Authority'))
(rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85076502'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#authoritativeLabel'), rdflib.term.Literal('Libraries', lang='en'))
(rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85076502'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#elementList'), rdflib.term.BNode('N38faf5ccd33a4ccdb9b323f3f149f862'))
(rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85076502'), rdflib.term.URIRef('http://id.loc.gov/ontologies/bflc/marcKey'), rdflib.term.Literal('150  $aLibraries'))
(rdflib.term.URIRe

In [7]:
for s, p, o in g:
    print(s)

http://id.loc.gov/authorities/subjects/sh85076502
N1a28595b4efe4013b57f230d00482db6
http://id.loc.gov/authorities/subjects/sh85076502
N1a28595b4efe4013b57f230d00482db6
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
N9f334144a5f447b98e40657a267e605c
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
Nb8678d7cb3634deb80b94b27e39d478b
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.gov/authorities/subjects/sh85076502
http://id.loc.

In [5]:
g.serialize('libraries.ttl', format='ttl')

<Graph identifier=N8c2571aa3f5546fdbfcdb568212cd7fa (<class 'rdflib.graph.Graph'>)>